In [1]:
import requests
from atlassian import Confluence
import time
from getpass import getpass

contentApiUrl = '/rest/api/content'
# Change these based on your instance
confluenceBaseUrl = 'https://confluence.corp.bioagilytix.com/confluence'


In [2]:

username = 'yangyang.liu'
password = getpass("password")
confluence = Confluence(
    url='https://confluence.corp.bioagilytix.com/confluence',
    username=username,
    password=password)


In [3]:
def getAllChildren(parentPageId):
    result = confluence.get_page_by_id(page_id=parentPageId, expand="children")
    ##requestUrl = result['children']['_links']['self']
    requestUrl = "{}{}/search?start=0&limit=9999&cql=parent={}".format(
        confluenceBaseUrl, contentApiUrl, "23005842")
    print(requestUrl)

    requestResponse = requests.get(requestUrl, auth=(username, password))
    print(requestResponse)
    return requestResponse.json()['results']


In [5]:
def isDraft(pageId):

    requestUrl = "https://confluence.corp.bioagilytix.com/confluence/rest/cw/1/content/{}/status".format(
        pageId)

    requestResponse = requests.get(requestUrl, auth=(username, password))
    if (requestResponse.status_code == 200):
        print(requestResponse.json())
        if requestResponse.json()['state']['name'] == "Draft":
            return True
    return False


In [6]:
def getConfiFormEntries(pageId, formName):
    requestUrl = "https://confluence.corp.bioagilytix.com/confluence/ajax/confiforms/rest/filter.action?pageId={}&f={}".format(pageId, formName)
    requestResponse = requests.get(requestUrl, auth=(username, password))
    
    return requestResponse.json()['list']['entry']
    


In [13]:
def getPageBody(pageId):
    requestUrl = "{}/rest/api/content/{}?expand=body.storage".format(
        confluenceBaseUrl, pageId)
    requestResponse = requests.get(requestUrl, auth=(username, password))
    return requestResponse.json()['body']['storage']['value']

In [14]:
bodyStorage = getPageBody(28607721)


In [33]:
from bs4 import BeautifulSoup



parsed_html = BeautifulSoup(bodyStorage)
form = parsed_html.find('ac:structured-macro', attrs={"ac:name": "confiform"})
print(form.findAll('ac:parameter', attrs={"ac:name": "formName"}))


[<ac:parameter ac:name="formName">runinfo</ac:parameter>, <ac:parameter ac:name="formName">runinfo</ac:parameter>, <ac:parameter ac:name="formName">runinfo</ac:parameter>]
